In [1]:
library(censusapi)
library(tidyr)
library(dplyr)
library(readr)
library(purrr)

ERROR: Error in library(censusapi): there is no package called ‘censusapi’


In [2]:
# get raw features -----------------------------------------------------------------------------------------------------------------

Sys.setenv(CENSUS_KEY="c8aa67e4086b4b5ce3a8717f59faa9a28f611dab")

# apis <- listCensusApis()
# View(apis)
# write.csv(apis, 'census_list.csv')

acs_vars_2019_5yr <- listCensusMetadata(name = "2019/acs/acs5", type='variables')
# saveRDS(acs_vars_2019_5yr, 'listCensusMetadata_2019acs5')
# acs_vars_2019_5yr <- readRDS("listCensusMetadata_2019acs5")

ERROR: Error in listCensusMetadata(name = "2019/acs/acs5", type = "variables"): could not find function "listCensusMetadata"


In [8]:
income <-
  getCensus(
    name = "acs/acs5",
    vintage = 2019,
    region = "zip code tabulation area:*", 
    regionin = "state:06",
    vars = "group(B06010)"
  ) %>% 
  select(c(zip_code_tabulation_area) | (ends_with(c("E"))) & !ends_with(c("EA","MA","M", "NAME", "state"))) %>% 
  select(c(zip_code_tabulation_area,
           B06010_002E, B06010_003E, B06010_004E, B06010_005E, B06010_006E, B06010_007E, B06010_008E, B06010_009E, B06010_010E, B06010_011E))

education <-
  getCensus(
    name = "acs/acs5",
    vintage = 2019,
    region = "zip code tabulation area:*", 
    regionin = "state:06",
    vars = "group(B06009)"
  ) %>% 
  select(c(zip_code_tabulation_area) | (ends_with(c("E"))) & !ends_with(c("EA","MA","M", "NAME", "state"))) %>% 
  select(c(zip_code_tabulation_area,
           B06009_002E, B06009_003E, B06009_004E, B06009_005E, B06009_006E))

sexAge <-
  getCensus(
    name = "acs/acs5",
    vintage = 2019,
    region = "zip code tabulation area:*", 
    regionin = "state:06",
    vars = "group(B01001)"
  ) %>% 
  select(c(zip_code_tabulation_area) | (ends_with(c("E"))) & !ends_with(c("EA","MA","M", "NAME", "state")))

# merge selected features
censusFeature <- income %>% 
      left_join(education, by = "zip_code_tabulation_area") %>% 
      left_join(sexAge, by = "zip_code_tabulation_area") %>% 
      arrange(zip_code_tabulation_area)
censusFeature <- censusFeature[, c('zip_code_tabulation_area', sort(colnames(censusFeature)))][, 1:ncol(censusFeature)]

In [9]:
# generate feature definitions
col <- colnames(censusFeature)[!colnames(censusFeature) %in% c('zip_code_tabulation_area')]
labels <- acs_vars_2019_5yr[acs_vars_2019_5yr$name %in% col, c('name', 'label')]
labels <- labels[order(labels$name), ]
# find definition details at https://api.census.gov/data/2019/acs/acs5/variables.html

# write.csv(censusFeature, '../data/census_features/labels.csv')

In [10]:
# get modified features -----------------------------------------------------------------------------------------------------------------
# get mean age
pop_male <- grepl('B01001_', labels$name) & grepl('!!Male', labels$label)
mean_male_age <- sapply(regmatches(labels[pop_male, ]$label, gregexpr("\\d+", labels[pop_male, ]$label)), function(x) mean(as.numeric(x)))
pop_female <- grepl('B01001_', labels$name) & grepl('!!Female', labels$label)
mean_female_age <- sapply(regmatches(labels[pop_female, ]$label, gregexpr("\\d+", labels[pop_female, ]$label)), function(x) mean(as.numeric(x)))
mean_pop_age <- sapply(c(NA, mean_male_age, mean_female_age), function(x) if(is.nan(x)) {x<-NA} else {x})

# get mean income
income_data <- grepl('B06010_', labels$name)
temp <- gsub(",", "", labels[income_data,]$label)
mean_income <- sapply(sapply(regmatches(temp, gregexpr("\\d+", temp)), function(x) mean(as.numeric(x))), function(x) if(is.nan(x)) {x<-NA} else {x})

# combine
mean_all <- c(mean_pop_age, rep(1, sum(grepl('B06009', labels$name))), mean_income) %>% replace_na(1)

pop_male[2] <- FALSE # change Male total population to False so that we don't adjust that column
pop_female[26] <- FALSE
income_data[55:56] <- rep(FALSE, 2)

censusFeature_mean <- censusFeature %>% 
  mutate_if(c(FALSE, pop_male), ~ ./censusFeature$B01001_002E) %>% 
  mutate_if(c(FALSE, pop_female), ~ ./censusFeature$B01001_026E) %>% 
  mutate_if(c(FALSE, income_data), ~ ./censusFeature$B01001_001E)

censusFeature_mean[, 2:ncol(censusFeature_mean)] <- sweep(censusFeature_mean[, 2:ncol(censusFeature_mean)], MARGIN=2, mean_all, `*`)

mean_male_age <- apply(censusFeature_mean[, 4:26], 1, function(x) sum(x, na.rm=TRUE))
mean_female_age <- apply(censusFeature_mean[, 28:50], 1, function(x) sum(x, na.rm=TRUE))
mean_income <- apply(censusFeature_mean[, 58:65], 1, function(x) sum(x, na.rm=TRUE))

censusFeature_mean <- censusFeature_mean %>% 
                      select(zip_code_tabulation_area | starts_with(c('B01001_001E', 'B01001_002E', 'B01001_026E', 'B06009', 'B06010_002E', 'B06010_003E'))) %>% 
                      mutate(mean_male_age=mean_male_age, mean_female_age=mean_female_age, mean_income=mean_income)
censusFeature_mean[censusFeature_mean$B01001_001E==0, c('mean_male_age', 'mean_female_age', 'mean_income')] <- NA

In [11]:
# convert to city level -----------------------------------------------------------------------------------------------------------------
# choose type of data to use
census_data <- censusFeature_mean #censusFeature or censusFeature_mean
colnames(census_data)[1] <- 'zipcode'

# load matching table and assign tract code based on `census_data` table
matching_table <- read.csv('../data/matching_cities_2.csv', header=TRUE)
matching_table <- matching_table %>% select(-1) %>% slice(-1) %>% 
                  mutate(afact=as.numeric(as.character(afact)),
                         afact2=as.numeric(as.character(afact2)),
                         zipcode=as.character(zipcode)) %>%
                  select(-c(census_block_group, afact, afact2)) %>%
                  distinct(city, zipcode)


# get mean of afacts from each tract_FIPS to a city
merged <- matching_table %>% left_join(select(census_data, 'zipcode' | starts_with("B") & ends_with(c("E")) | starts_with('mean_')), by='zipcode') %>% arrange(city, zipcode)

# weight mean feature in the same city by population density (only do this for `censusFeature_mean`)
all_pop <- merged %>% group_by(city) %>% summarise(city_pop=sum(B01001_001E))
pop_den <- merged %>% 
           mutate(pop_den=sapply(seq(length(zipcode)), function(i) B01001_001E[i]/all_pop[all_pop$city==city[i], ]$city_pop)) %>%
           select(zipcode, city, pop_den) %>% arrange(city, zipcode)
average_col <- grepl('mean', colnames(merged)) #| grepl('B06010_002E', colnames(merged)) | grepl('B06010_003E', colnames(merged))
merged[, average_col] <- sweep(merged[, average_col], MARGIN=1, pop_den$pop_den, `*`)

# final aggregation and fill minor NAs
merged_final <- merged %>% group_by(city) %>% summarise(across(where(is.numeric), sum)) %>% 
                mutate_if(is.numeric, list(~ ifelse(is.na(.), mean(., na.rm = TRUE), .)))


#write.csv(merged_final, '../data/census_features/censusFeature_mean.csv')
# write.csv(merged_final, '../data/census_features/censusFeature.csv')

`summarise()` ungrouping output (override with `.groups` argument)

`summarise()` ungrouping output (override with `.groups` argument)



In [24]:
merged_final = merged_final[,c(1,12:14)]

In [26]:
write.csv(merged_final, '/home/mxenoc/workspace/covid-CA-forecasting/data/censusFeature_mean.csv')

In [1]:
getwd()

[1] "/home/mxenoc/workspace/covid-CA-forecasting/notebooks"

In [2]:
library(censusapi)
library(tidyr)
library(dplyr)
library(readr)
library(purrr)

# get raw features -----------------------------------------------------------------------------------------------------------------

Sys.setenv(CENSUS_KEY="c8aa67e4086b4b5ce3a8717f59faa9a28f611dab")

# apis <- listCensusApis()
# View(apis)
# write.csv(apis, 'census_list.csv')

acs_vars_2019_5yr <- listCensusMetadata(name = "2019/acs/acs5", type='variables')
# saveRDS(acs_vars_2019_5yr, 'listCensusMetadata_2019acs5')
# acs_vars_2019_5yr <- readRDS("listCensusMetadata_2019acs5")


Attaching package: ‘censusapi’


The following object is masked from ‘package:methods’:

    getFunction



Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [3]:
income <-
  getCensus(
    name = "acs/acs5",
    vintage = 2019,
    region = "zip code tabulation area:*", 
    regionin = "state:06",
    vars = "group(B06010)"
  ) %>% 
  select(c(zip_code_tabulation_area) | (ends_with(c("E"))) & !ends_with(c("EA","MA","M", "NAME", "state"))) %>% 
  select(c(zip_code_tabulation_area,
           B06010_002E, B06010_003E, B06010_004E, B06010_005E, B06010_006E, B06010_007E, B06010_008E, B06010_009E, B06010_010E, B06010_011E))

education <-
  getCensus(
    name = "acs/acs5",
    vintage = 2019,
    region = "zip code tabulation area:*", 
    regionin = "state:06",
    vars = "group(B06009)"
  ) %>% 
  select(c(zip_code_tabulation_area) | (ends_with(c("E"))) & !ends_with(c("EA","MA","M", "NAME", "state"))) %>% 
  select(c(zip_code_tabulation_area,
           B06009_002E, B06009_003E, B06009_004E, B06009_005E, B06009_006E))

sexAge <-
  getCensus(
    name = "acs/acs5",
    vintage = 2019,
    region = "zip code tabulation area:*", 
    regionin = "state:06",
    vars = "group(B01001)"
  ) %>% 
  select(c(zip_code_tabulation_area) | (ends_with(c("E"))) & !ends_with(c("EA","MA","M", "NAME", "state")))


pphh <-
  getCensus(
    name = "acs/acs5",
    vintage = 2019,
    region = "zip code tabulation area:*", 
    regionin = "state:06",
    vars = "group(B08201)"
  ) %>% 
  select(c(zip_code_tabulation_area) | (ends_with(c("E"))) & !ends_with(c("EA","MA","M", "NAME", "state"))) %>% 
  select(c(zip_code_tabulation_area,
           B08201_001E, B08201_007E, B08201_013E, B08201_019E, B08201_025E))

race <-
  getCensus(
    name = "acs/acs5",
    vintage = 2019,
    region = "zip code tabulation area:*", 
    regionin = "state:06",
    vars = "group(B02001)"
  ) %>% 
  select(c(zip_code_tabulation_area) | (ends_with(c("E"))) & !ends_with(c("EA","MA","M", "NAME", "state", "001E")))

citizenship <-
  getCensus(
    name = "acs/acs5",
    vintage = 2019,
    region = "zip code tabulation area:*", 
    regionin = "state:06",
    vars = "group(B05001)"
  ) %>% 
  select(c(zip_code_tabulation_area) | (ends_with(c("E"))) & !ends_with(c("EA","MA","M", "NAME", "state"))) 

hisByOri <-
  getCensus(
    name = "acs/acs5",
    vintage = 2019,
    region = "zip code tabulation area:*", 
    regionin = "state:06",
    vars = "group(B03001)"
  ) %>% 
  select(c(zip_code_tabulation_area) | (ends_with(c("E"))) & !ends_with(c("EA","MA","M", "NAME", "state"))) 

hisByRace <-
  getCensus(
    name = "acs/acs5",
    vintage = 2019,
    region = "zip code tabulation area:*", 
    regionin = "state:06",
    vars = "group(B03002)"
  ) %>% 
  select(c(zip_code_tabulation_area) | (ends_with(c("E"))) & !ends_with(c("EA","MA","M", "NAME", "state"))) 

incToPoverty <-
  getCensus(
    name = "acs/acs5",
    vintage = 2019,
    region = "zip code tabulation area:*", 
    regionin = "state:06",
    vars = "group(B05010)"
  ) %>% 
  select(c(zip_code_tabulation_area) | (ends_with(c("E"))) & !ends_with(c("EA","MA","M", "NAME", "state"))) 

placeOfBirthBySex <-
  getCensus(
    name = "acs/acs5",
    vintage = 2019,
    region = "zip code tabulation area:*", 
    regionin = "state:06",
    vars = "group(B06003)"
  ) %>% 
  select(c(zip_code_tabulation_area) | (ends_with(c("E"))) & !ends_with(c("EA","MA","M", "NAME", "state", 
        "001E", "002E", "003E"))) 

placeOfBirthByPoverty <-
  getCensus(
    name = "acs/acs5",
    vintage = 2019,
    region = "zip code tabulation area:*", 
    regionin = "state:06",
    vars = "group(B06012)"
  ) %>% 
  select(c(zip_code_tabulation_area) | (ends_with(c("E"))) & !ends_with(c("EA","MA","M", "NAME", "state"))) 

speak <-
  getCensus(
    name = "acs/acs5",
    vintage = 2019,
    region = "zip code tabulation area:*", 
    regionin = "state:06",
    vars = "group(B06007)"
  ) %>% 
  select(c(zip_code_tabulation_area) | (ends_with(c("E"))) & !ends_with(c("EA","MA","M", "NAME", "state")))  %>% 
  select(c(zip_code_tabulation_area,
           B06007_002E, B06007_003E, B06007_006E))

mobilByAge <-
  getCensus(
    name = "acs/acs5",
    vintage = 2019,
    region = "zip code tabulation area:*", 
    regionin = "state:06",
    vars = "group(B07001)"
  ) %>% 
  select(c(zip_code_tabulation_area) | (ends_with(c("E"))) & !ends_with(c("EA","MA","M", "NAME", "state"))) 

mobilByIncome <-
  getCensus(
    name = "acs/acs5",
    vintage = 2019,
    region = "zip code tabulation area:*", 
    regionin = "state:06",
    vars = "group(B07010)"
  ) %>% 
  select(c(zip_code_tabulation_area) | (ends_with(c("E"))) & !ends_with(c("EA","MA","M", "NAME", "state"))) 

# mobilByResidence <-
#   getCensus(
#     name = "acs/acs5",
#     vintage = 2019,
#     region = "zip code tabulation area:*", 
#     regionin = "state:06",
#     vars = "group(B07401)"
#   ) %>% 
#   select(c(zip_code_tabulation_area) | (ends_with(c("E"))) & !ends_with(c("EA","MA","M", "NAME", "state"))) 

transportSex <-
  getCensus(
    name = "acs/acs5",
    vintage = 2019,
    region = "zip code tabulation area:*", 
    regionin = "state:06",
    vars = "group(B08006)"
  ) %>% 
  select(c(zip_code_tabulation_area) | (ends_with(c("E"))) & !ends_with(c("EA","MA","M", "NAME", "state"))) %>%
  select(c(zip_code_tabulation_area,
           B08006_002E, B08006_008E, B08006_014E, B08006_015E, B08006_016E, B08006_017E))

placeOfWork <-
  getCensus(
    name = "acs/acs5",
    vintage = 2019,
    region = "zip code tabulation area:*", 
    regionin = "state:06",
    vars = "group(B08007)"
  ) %>% 
  select(c(zip_code_tabulation_area) | (ends_with(c("E"))) & !ends_with(c("EA","MA","M", "NAME", "state"))) %>%
  select(c(zip_code_tabulation_area,
           B08007_002E, B08007_003E, B08007_004E, B08007_005E)) 

departureTime <-
  getCensus(
    name = "acs/acs5",
    vintage = 2019,
    region = "zip code tabulation area:*", 
    regionin = "state:06",
    vars = "group(B08011)"
  ) %>% 
  select(c(zip_code_tabulation_area) | (ends_with(c("E"))) & !ends_with(c("EA","MA","M", "NAME", "state"))) %>%
  select(c(zip_code_tabulation_area,
           B08011_002E, B08011_003E, B08011_004E, B08011_005E,
           B08011_006E, B08011_007E, B08011_008E, B08011_009E, B08011_010E,
           B08011_011E, B08011_012E, B08011_013E, B08011_014E, B08011_015E))

travelTime <-
  getCensus(
    name = "acs/acs5",
    vintage = 2019,
    region = "zip code tabulation area:*", 
    regionin = "state:06",
    vars = "group(B08012)"
  ) %>% 
  select(c(zip_code_tabulation_area) | (ends_with(c("E"))) & !ends_with(c("EA","MA","M", "NAME", "state"))) %>%
  select(c(zip_code_tabulation_area,
           B08012_002E, B08012_003E, B08012_004E, B08012_005E,
           B08012_006E, B08012_007E, B08012_008E, B08012_009E, B08012_010E,
           B08012_011E, B08012_012E, B08012_013E))

transportByAge <-
  getCensus(
    name = "acs/acs5",
    vintage = 2019,
    region = "zip code tabulation area:*", 
    regionin = "state:06",
    vars = "group(B08101)"
  ) %>% 
  select(c(zip_code_tabulation_area) | (ends_with(c("E"))) & !ends_with(c("EA","MA","M", "NAME", "state"))) 

transportByPoverty <-
  getCensus(
    name = "acs/acs5",
    vintage = 2019,
    region = "zip code tabulation area:*", 
    regionin = "state:06",
    vars = "group(B08122)"
  ) %>% 
  select(c(zip_code_tabulation_area) | (ends_with(c("E"))) & !ends_with(c("EA","MA","M", "NAME", "state"))) 

transportByOccupation <-
  getCensus(
    name = "acs/acs5",
    vintage = 2019,
    region = "zip code tabulation area:*", 
    regionin = "state:06",
    vars = "group(B08124)"
  ) %>% 
  select(c(zip_code_tabulation_area) | (ends_with(c("E"))) & !ends_with(c("EA","MA","M", "NAME", "state"))) 

transportByIndustry <-
  getCensus(
    name = "acs/acs5",
    vintage = 2019,
    region = "zip code tabulation area:*", 
    regionin = "state:06",
    vars = "group(B08126)"
  ) %>% 
  select(c(zip_code_tabulation_area) | (ends_with(c("E"))) & !ends_with(c("EA","MA","M", "NAME", "state"))) 

transportByClassWorker <-
  getCensus(
    name = "acs/acs5",
    vintage = 2019,
    region = "zip code tabulation area:*", 
    regionin = "state:06",
    vars = "group(B08128)"
  ) %>% 
  select(c(zip_code_tabulation_area) | (ends_with(c("E"))) & !ends_with(c("EA","MA","M", "NAME", "state"))) 

hhByNumWorker <-
  getCensus(
    name = "acs/acs5",
    vintage = 2019,
    region = "zip code tabulation area:*", 
    regionin = "state:06",
    vars = "group(B08202)"
  ) %>% 
  select(c(zip_code_tabulation_area) | (ends_with(c("E"))) & !ends_with(c("EA","MA","M", "NAME", "state"))) 

transport <-
  getCensus(
    name = "acs/acs5",
    vintage = 2019,
    region = "zip code tabulation area:*", 
    regionin = "state:06",
    vars = "group(B08105A)"
  ) %>% 
  select(c(zip_code_tabulation_area) | (ends_with(c("E"))) & !ends_with(c("EA","MA","M", "NAME", "state"))) 

# transportByEarning <-
#   getCensus(
#     name = "acs/acs5",
#     vintage = 2019,
#     region = "zip code tabulation area:*", 
#     regionin = "state:06",
#     vars = "group(B08519)"
#   ) %>% 
#   select(c(zip_code_tabulation_area) | (ends_with(c("E"))) & !ends_with(c("EA","MA","M", "NAME", "state"))) 

hhType <-
  getCensus(
    name = "acs/acs5",
    vintage = 2019,
    region = "zip code tabulation area:*", 
    regionin = "state:06",
    vars = "group(B11001)"
  ) %>% 
  select(c(zip_code_tabulation_area) | (ends_with(c("E"))) & !ends_with(c("EA","MA","M", "NAME", "state"))) 

hhTypeByRelatives <-
  getCensus(
    name = "acs/acs5",
    vintage = 2019,
    region = "zip code tabulation area:*", 
    regionin = "state:06",
    vars = "group(B11002)"
  ) %>% 
  select(c(zip_code_tabulation_area) | (ends_with(c("E"))) & !ends_with(c("EA","MA","M", "NAME", "state"))) 

hhTypeByUnits <-
  getCensus(
    name = "acs/acs5",
    vintage = 2019,
    region = "zip code tabulation area:*", 
    regionin = "state:06",
    vars = "group(B11011)"
  ) %>% 
  select(c(zip_code_tabulation_area) | (ends_with(c("E"))) & !ends_with(c("EA","MA","M", "NAME", "state"))) 

hhTypeByPresenceOfNonRelatives <-
  getCensus(
    name = "acs/acs5",
    vintage = 2019,
    region = "zip code tabulation area:*", 
    regionin = "state:06",
    vars = "group(B11015)"
  ) %>% 
  select(c(zip_code_tabulation_area) | (ends_with(c("E"))) & !ends_with(c("EA","MA","M", "NAME", "state"))) 

edu <-
  getCensus(
    name = "acs/acs5",
    vintage = 2019,
    region = "zip code tabulation area:*", 
    regionin = "state:06",
    vars = "group(B15003)"
  ) %>% 
  select(c(zip_code_tabulation_area) | (ends_with(c("E"))) & !ends_with(c("EA","MA","M", "NAME", "state"))) 

sexByMajor <-
  getCensus(
    name = "acs/acs5",
    vintage = 2019,
    region = "zip code tabulation area:*", 
    regionin = "state:06",
    vars = "group(B15011)"
  ) %>% 
  select(c(zip_code_tabulation_area) | (ends_with(c("E"))) & !ends_with(c("EA","MA","M", "NAME", "state"))) 

# languageAtHome <-
#   getCensus(
#     name = "acs/acs5",
#     vintage = 2019,
#     region = "zip code tabulation area:*", 
#     regionin = "state:06",
#     vars = "group(B16001)"
#   ) %>% 
#   select(c(zip_code_tabulation_area) | (ends_with(c("E"))) & !ends_with(c("EA","MA","M", "NAME", "state"))) 

povertyByAge <-
  getCensus(
    name = "acs/acs5",
    vintage = 2019,
    region = "zip code tabulation area:*", 
    regionin = "state:06",
    vars = "group(B16009)"
  ) %>% 
  select(c(zip_code_tabulation_area) | (ends_with(c("E"))) & !ends_with(c("EA","MA","M", "NAME", "state"))) 

povertyBySex <-
  getCensus(
    name = "acs/acs5",
    vintage = 2019,
    region = "zip code tabulation area:*", 
    regionin = "state:06",
    vars = "group(B17001F)"
  ) %>% 
  select(c(zip_code_tabulation_area) | (ends_with(c("E"))) & !ends_with(c("EA","MA","M", "NAME", "state"))) 

povertyByEmployment <-
  getCensus(
    name = "acs/acs5",
    vintage = 2019,
    region = "zip code tabulation area:*", 
    regionin = "state:06",
    vars = "group(B17005)"
  ) %>% 
  select(c(zip_code_tabulation_area) | (ends_with(c("E"))) & !ends_with(c("EA","MA","M", "NAME", "state"))) 

disability <-
  getCensus(
    name = "acs/acs5",
    vintage = 2019,
    region = "zip code tabulation area:*", 
    regionin = "state:06",
    vars = "group(B19051)"
  ) %>% 
  select(c(zip_code_tabulation_area) | (ends_with(c("E"))) & !ends_with(c("EA","MA","M", "NAME", "state"))) 

wage <-
  getCensus(
    name = "acs/acs5",
    vintage = 2019,
    region = "zip code tabulation area:*", 
    regionin = "state:06",
    vars = "group(B19052)"
  ) %>% 
  select(c(zip_code_tabulation_area) | (ends_with(c("E"))) & !ends_with(c("EA","MA","M", "NAME", "state"))) 

familyIncome <-
  getCensus(
    name = "acs/acs5",
    vintage = 2019,
    region = "zip code tabulation area:*", 
    regionin = "state:06",
    vars = "group(B19101)"
  ) %>% 
  select(c(zip_code_tabulation_area) | (ends_with(c("E"))) & !ends_with(c("EA","MA","M", "NAME", "state"))) 

numChildren <-
  getCensus(
    name = "acs/acs5",
    vintage = 2019,
    region = "zip code tabulation area:*", 
    regionin = "state:06",
    vars = "group(B19131)"
  ) %>% 
  select(c(zip_code_tabulation_area) | (ends_with(c("E"))) & !ends_with(c("EA","MA","M", "NAME", "state"))) 

workexp <-
  getCensus(
    name = "acs/acs5",
    vintage = 2019,
    region = "zip code tabulation area:*", 
    regionin = "state:06",
    vars = "group(B19325)"
  ) %>% 
  select(c(zip_code_tabulation_area) | (ends_with(c("E"))) & !ends_with(c("EA","MA","M", "NAME", "state"))) 

foodstamp <-
  getCensus(
    name = "acs/acs5",
    vintage = 2019,
    region = "zip code tabulation area:*", 
    regionin = "state:06",
    vars = "group(B22001)"
  ) %>% 
  select(c(zip_code_tabulation_area) | (ends_with(c("E"))) & !ends_with(c("EA","MA","M", "NAME", "state"))) 

occupationByEarnings <-
  getCensus(
    name = "acs/acs5",
    vintage = 2019,
    region = "zip code tabulation area:*", 
    regionin = "state:06",
    vars = "group(B24011)"
  ) %>% 
  select(c(zip_code_tabulation_area) | (ends_with(c("E"))) & !ends_with(c("EA","MA","M", "NAME", "state"))) 

# occupationFemale <-
#   getCensus(
#     name = "acs/acs5",
#     vintage = 2019,
#     region = "zip code tabulation area:*", 
#     regionin = "state:06",
#     vars = "group(B24116)"
#   ) %>% 
#   select(c(zip_code_tabulation_area) | (ends_with(c("E"))) & !ends_with(c("EA","MA","M", "NAME", "state"))) 

occupantPerRoom <-
  getCensus(
    name = "acs/acs5",
    vintage = 2019,
    region = "zip code tabulation area:*", 
    regionin = "state:06",
    vars = "group(B25014)"
  ) %>% 
  select(c(zip_code_tabulation_area) | (ends_with(c("E"))) & !ends_with(c("EA","MA","M", "NAME", "state"))) 

occupantPerRoomHisp <-
  getCensus(
    name = "acs/acs5",
    vintage = 2019,
    region = "zip code tabulation area:*", 
    regionin = "state:06",
    vars = "group(B25014I)"
  ) %>% 
  select(c(zip_code_tabulation_area) | (ends_with(c("E"))) & !ends_with(c("EA","MA","M", "NAME", "state"))) 

rooms <-
  getCensus(
    name = "acs/acs5",
    vintage = 2019,
    region = "zip code tabulation area:*", 
    regionin = "state:06",
    vars = "group(B25017)"
  ) %>% 
  select(c(zip_code_tabulation_area) | (ends_with(c("E"))) & !ends_with(c("EA","MA","M", "NAME", "state"))) 

numUnits <-
  getCensus(
    name = "acs/acs5",
    vintage = 2019,
    region = "zip code tabulation area:*", 
    regionin = "state:06",
    vars = "group(B25024)"
  ) %>% 
  select(c(zip_code_tabulation_area) | (ends_with(c("E"))) & !ends_with(c("EA","MA","M", "NAME", "state"))) 

insuranceByAge <-
  getCensus(
    name = "acs/acs5",
    vintage = 2019,
    region = "zip code tabulation area:*", 
    regionin = "state:06",
    vars = "group(B27001)"
  ) %>% 
  select(c(zip_code_tabulation_area) | (ends_with(c("E"))) & !ends_with(c("EA","MA","M", "NAME", "state"))) 

insuranceByEmployment <-
  getCensus(
    name = "acs/acs5",
    vintage = 2019,
    region = "zip code tabulation area:*", 
    regionin = "state:06",
    vars = "group(B27011)"
  ) %>% 
  select(c(zip_code_tabulation_area) | (ends_with(c("E"))) & !ends_with(c("EA","MA","M", "NAME", "state"))) 

detailedRace <-
  getCensus(
    name = "acs/acs5",
    vintage = 2019,
    region = "zip code tabulation area:*", 
    regionin = "state:06",
    vars = "group(C02003)"
  ) %>% 
  select(c(zip_code_tabulation_area) | (ends_with(c("E"))) & !ends_with(c("EA","MA","M", "NAME", "state"))) 

industryByWorker <-
  getCensus(
    name = "acs/acs5",
    vintage = 2019,
    region = "zip code tabulation area:*", 
    regionin = "state:06",
    vars = "group(C24070)"
  ) %>% 
  select(c(zip_code_tabulation_area) | (ends_with(c("E"))) & !ends_with(c("EA","MA","M", "NAME", "state"))) 


In [4]:
all_var <- ls()[!ls() %in% c("acs_vars_2019_5yr", "all_var", "aaa", "col", "labels", "censusFeature")]

# merge selected features
censusFeature <- lapply(all_var, function(x) eval(as.name(x))) %>% 
                 reduce(left_join, , by = "zip_code_tabulation_area") %>%
                 arrange(zip_code_tabulation_area)
censusFeature <- censusFeature[, c('zip_code_tabulation_area', sort(colnames(censusFeature)))][, 1:ncol(censusFeature)]

In [5]:
# generate feature definitions
col <- colnames(censusFeature)[!colnames(censusFeature) %in% c('zip_code_tabulation_area')]
labels <- acs_vars_2019_5yr[acs_vars_2019_5yr$name %in% col, c('name', 'label')]
labels <- labels[order(labels$name), ]
# find definition details at https://api.census.gov/data/2019/acs/acs5/variables.html

# write.csv(censusFeature, '../data/census_features/labels.csv')

In [6]:
# get modified features -----------------------------------------------------------------------------------------------------------------
get_mean <- function (x) {
  temp <- gsub(",", "", labels[x, ]$label)
  means <-  sapply(sapply(regmatches(temp, gregexpr("\\d+", temp)), 
            function(x) mean(as.numeric(x))), function(x) if(is.nan(x)) {x<-NA} else {x})
  return (means)
}

# pop_male <- grepl('B01001_', labels$name) & grepl('!!Male', labels$label)
# mean_male_age <- sapply(regmatches(labels[pop_male, ]$label, gregexpr("\\d+", labels[pop_male, ]$label)), function(x) mean(as.numeric(x)))
# pop_female <- grepl('B01001_', labels$name) & grepl('!!Female', labels$label)
# mean_female_age <- sapply(regmatches(labels[pop_female, ]$label, gregexpr("\\d+", labels[pop_female, ]$label)), function(x) mean(as.numeric(x)))
# mean_pop_age <- sapply(c(NA, mean_male_age, mean_female_age), function(x) if(is.nan(x)) {x<-NA} else {x})

# # get mean income
# income_data <- grepl('B06010_', labels$name)
# temp <- gsub(",", "", labels[income_data,]$label)
# mean_income <- sapply(sapply(regmatches(temp, gregexpr("\\d+", temp)), function(x) mean(as.numeric(x))), function(x) if(is.nan(x)) {x<-NA} else {x})

# # get mean person per household
# pphh_data <- grepl('B08201_', labels$name)
# temp <- gsub(",", "", labels[pphh_data,]$label)
# mean_pphh <- sapply(sapply(regmatches(temp, gregexpr("\\d+", temp)), function(x) mean(as.numeric(x))), function(x) if(is.nan(x)) {x<-NA} else {x})

# B05010 <- get_mean(colnames(censusFeature) %in% c('B05010_002E', 'B05010_010E', 'B05010_018E'))
# B07001 <- get_mean(colnames(censusFeature) %in% c(paste0('B07001_00', seq(2,9), 'E'), paste0('B07001_0', seq(10,15), 'E')))
# B07010 <- get_mean(colnames(censusFeature) %in% c(paste0('B07010_00', seq(4,9), 'E'), paste0('B07010_0', seq(10,11), 'E')))
# B08012 <- get_mean(colnames(censusFeature) %in% c(paste0('B08012_00', seq(2,9), 'E'), paste0('B08012_0', seq(10,13), 'E')))
# B08202_2 <- get_mean(colnames(censusFeature) %in% c('B08202_006E', 'B08202_009E', 'B08202_013E', 'B08202_018E'))
# B11015 <- get_mean(colnames(censusFeature) %in% c('B11015_002E', 'B11015_003E'))

In [7]:
labels[grepl('No workers', labels$label), 'label'] <- gsub('No workers', '0 workers', labels[grepl('No workers', labels$label), 'label'])

pop <- grepl('B01001_', labels$name)
mean_pop_age <- get_mean(pop)
mean_pop_age[!grepl('!!Female|!!Male', labels[grepl('B01001_', labels$name), ]$label)] <- NA

income_data <- grepl('B06010_', labels$name)
mean_income <- get_mean(income_data)

pphh_data <- grepl('B08201_', labels$name)
mean_pphh <- get_mean(pphh_data)

B08202_1 <- grepl('B08202_', labels$name)
mean_B08202_1 <- get_mean(B08202_1)
mean_B08202_1[grepl('person', labels[grepl('B08202_', labels$name), ]$label)] <- NA

# B17001F <- grepl('B17001F_', labels$name)
# mean_B17001F <- get_mean(B17001F)
# mean_B17001F[!grepl('!!Male:!!|!!Female:!!', labels[grepl('B17001F_', labels$name), ]$label)] <- NA

B19101 <- grepl('B19101_', labels$name)
mean_B19101 <- get_mean(B19101)

B25014 <- grepl('B25014_', labels$name)
mean_B25014 <- get_mean(B25014)
mean_B25014[!grepl('!!Owner occupied:!!|!!Renter occupied:!!', labels[grepl('B25014_', labels$name), ]$label)] <- NA

B25017 <- grepl('B25017_', labels$name)
mean_B25017 <- get_mean(B25017)

In [8]:
# combine
mean_all <- c(mean_pop_age, rep(1, sum(grepl('B06009', labels$name))), mean_income, mean_pphh,
              mean_B08202_1, mean_B19101, mean_B25014, mean_B25017) %>% replace_na(1)

# pop_male[2] <- FALSE # change Male total population to False so that we don't adjust that column
# pop_female[26] <- FALSE
# income_data[55:56] <- rep(FALSE, 2)
# pphh_data[65] <- FALSE
# B08202_1[B08202_1][is.na(mean_B08202_1)] <- FALSE
# B17001F_1[B17001F_1][is.na(mean_B17001F_1)] <- FALSE
# B17001F_2[B17001F_2][is.na(mean_B17001F_2)] <- FALSE
# B19101[B19101][is.na(mean_B19101)] <- FALSE
# B25014_1[B25014_1][is.na(mean_B25014_1)] <- FALSE
# B25014_2[B25014_2][is.na(mean_B25014_2)] <- FALSE
# B25017[B25017][is.na(mean_B25017)] <- FALSE

sel_col <- c('zip_code_tabulation_area', 'B01001_', 'B06010_', 'B08201_', 'B08202_', 'B06009_', 'B19101_', 'B25014_', 'B25017_')
censusFeature_mean <- censusFeature %>% select(starts_with(sel_col))
new_label <- labels[grepl(paste0(sel_col, collapse='|'), labels$name), ]

censusFeature_mean <- censusFeature_mean %>% 
  mutate_if(c(FALSE, grepl('B01001_', new_label$name) & grepl('!!Male', new_label$label) & grepl('\\d', new_label$label)), ~ ./censusFeature$B01001_002E) %>%
  mutate_if(c(FALSE, grepl('B01001_', new_label$name) & grepl('!!Female', new_label$label) & grepl('\\d', new_label$label)), ~ ./censusFeature$B01001_026E) %>%
  mutate_if(c(FALSE, grepl('B06010_', new_label$name) & grepl('\\d', new_label$label)), ~ ./censusFeature$B01001_001E) %>%
  mutate_if(c(FALSE, grepl('B08201_', new_label$name) & grepl('\\d', new_label$label)), ~ ./censusFeature$B08201_001E) %>%
  mutate_if(c(FALSE, grepl('B08202_', new_label$name) & grepl('\\d', new_label$label)), ~ ./censusFeature$B08202_001E) %>%
  mutate_if(c(FALSE, grepl('B19101_', new_label$name) & grepl('\\d', new_label$label)), ~ ./censusFeature$B19101_001E) %>%
  mutate_if(c(FALSE, grepl('B25014_', new_label$name) & grepl('!!Owner occupied:!!', new_label$label) & grepl('\\d', new_label$label)), ~ ./censusFeature$B01001_002E) %>%
  mutate_if(c(FALSE, grepl('B25014_', new_label$name) & grepl('!!Renter occupied:!!', new_label$label) & grepl('\\d', new_label$label)), ~ ./censusFeature$B01001_002E) %>%
  mutate_if(c(FALSE, grepl('B25017_', new_label$name) & grepl('\\d', new_label$label)), ~ ./censusFeature$B25017_001E)
  
  # mutate_if(c(FALSE, pop_female), ~ ./censusFeature$B01001_002E) %>% 
  # mutate_if(c(FALSE, pop_female), ~ ./censusFeature$B01001_026E) %>% 
  # mutate_if(c(FALSE, income_data), ~ ./censusFeature$B01001_001E) %>% 
  # mutate_if(c(FALSE, pphh_data), ~ ./censusFeature$B08201_001E) %>% 
  # mutate_if(c(FALSE, B08202_1), ~ ./censusFeature$B08202_001E) %>% 
  # mutate_if(c(FALSE, B17001F_1), ~ ./censusFeature$B17001F_001E) %>% 
  # mutate_if(c(FALSE, B17001F_2), ~ ./censusFeature$B17001F_001E) %>% 
  # mutate_if(c(FALSE, B19101), ~ ./censusFeature$B19101_001E) %>% 
  # mutate_if(c(FALSE, B25014_1), ~ ./censusFeature$B25014_001E) %>% 
  # mutate_if(c(FALSE, B25014_2), ~ ./censusFeature$B25014_001E) %>% 
  # mutate_if(c(FALSE, B25017), ~ ./censusFeature$B25017_001E)

# censusFeature_mean[is.na(censusFeature_mean)] <- replace_na(0)
censusFeature_mean[, 2:ncol(censusFeature_mean)] <- sweep(censusFeature_mean[, 2:ncol(censusFeature_mean)], MARGIN=2, mean_all, `*`)

mean_male_age <- apply(censusFeature_mean[c(FALSE, grepl('B01001_', new_label$name) & grepl('!!Male', new_label$label) & grepl('\\d', new_label$label))], 1, function(x) sum(x, na.rm=TRUE))
mean_female_age <- apply(censusFeature_mean[c(FALSE, grepl('B01001_', new_label$name) & grepl('!!Female', new_label$label) & grepl('\\d', new_label$label))], 1, function(x) sum(x, na.rm=TRUE))
mean_income <- apply(censusFeature_mean[c(FALSE, grepl('B06010_', new_label$name) & grepl('\\d', new_label$label))], 1, function(x) sum(x, na.rm=TRUE))
mean_pphh <- apply(censusFeature_mean[c(FALSE, grepl('B08201_', new_label$name) & grepl('\\d', new_label$label))], 1, function(x) sum(x, na.rm=TRUE))
mean_B08202_1 <- apply(censusFeature_mean[c(FALSE, grepl('B08202_', new_label$name) & grepl('\\d', new_label$label))], 1, function(x) sum(x, na.rm=TRUE))
mean_B19101 <- apply(censusFeature_mean[c(FALSE, grepl('B19101_', new_label$name) & grepl('\\d', new_label$label))], 1, function(x) sum(x, na.rm=TRUE))
mean_B25014_1 <- apply(censusFeature_mean[c(FALSE, grepl('B25014_', new_label$name) & grepl('!!Owner occupied:!!', new_label$label) & grepl('\\d', new_label$label))], 1, function(x) sum(x, na.rm=TRUE))
mean_B25014_2 <- apply(censusFeature_mean[c(FALSE, grepl('B25014_', new_label$name) & grepl('!!Renter occupied:!!', new_label$label) & grepl('\\d', new_label$label))], 1, function(x) sum(x, na.rm=TRUE))
mean_B25017 <- apply(censusFeature_mean[c(FALSE, grepl('B25017_', new_label$name) & grepl('\\d', new_label$label))], 1, function(x) sum(x, na.rm=TRUE))



censusFeature_mean <- censusFeature_mean %>% 
                      select(zip_code_tabulation_area | starts_with(c('B01001_001E', 'B01001_002E', 'B01001_026E', 'B06009', 'B06010_002E', 'B06010_003E', 'B08201_001E',
                                                                      'B08202_001E', 'B19101_001E', 'B01001_002E', 'B25017_001E'))) %>% 
                      mutate(mean_male_age=mean_male_age, mean_female_age=mean_female_age, mean_income=mean_income, mean_pphh=mean_pphh,
                             mean_B08202=mean_B08202_1, mean_B19101=mean_B19101, mean_B25014_owner=mean_B25014_1, mean_B25014_renter=mean_B25014_2,
                             mean_B25017=mean_B25017)
censusFeature_mean[censusFeature_mean$B01001_001E==0, grepl('mean_', colnames(censusFeature_mean))] <- NA

In [9]:
# convert to city level -----------------------------------------------------------------------------------------------------------------
# choose type of data to use
census_data <- censusFeature_mean #censusFeature or censusFeature_mean
colnames(census_data)[1] <- 'zipcode'

In [10]:
# load matching table and assign tract code based on `census_data` table
matching_table <- read.csv('../data/matching_cities_2.csv', header=TRUE)
matching_table <- matching_table %>% select(-1) %>% slice(-1) %>% 
                  mutate(afact=as.numeric(as.character(afact)),
                         afact2=as.numeric(as.character(afact2)),
                         zipcode=as.character(zipcode)) %>%
                  select(-c(census_block_group, afact, afact2)) %>%
                  distinct(city, zipcode)


# get mean of afacts from each tract_FIPS to a city
merged <- matching_table %>% left_join(select(census_data, 'zipcode' | starts_with("B") & ends_with(c("E")) | starts_with('mean_')), by='zipcode') %>% arrange(city, zipcode)

Warning message in file(file, "rt"):
“cannot open file '../data/matching_cities_2.csv': No such file or directory”


ERROR: Error in file(file, "rt"): cannot open the connection


In [11]:
# weight mean feature in the same city by population density (only do this for `censusFeature_mean`)
all_pop <- merged %>% group_by(city) %>% summarise(city_pop=sum(B01001_001E))
pop_den <- merged %>% 
           mutate(pop_den=sapply(seq(length(zipcode)), function(i) B01001_001E[i]/all_pop[all_pop$city==city[i], ]$city_pop)) %>%
           select(zipcode, city, pop_den) %>% arrange(city, zipcode)
average_col <- grepl('mean', colnames(merged)) #| grepl('B06010_002E', colnames(merged)) | grepl('B06010_003E', colnames(merged))
merged[, average_col] <- sweep(merged[, average_col], MARGIN=1, pop_den$pop_den, `*`)

# final aggregation and fill minor NAs
merged_final <- merged %>% group_by(city) %>% summarise(across(where(is.numeric), sum)) %>% 
                mutate_if(is.numeric, list(~ ifelse(is.na(.), mean(., na.rm = TRUE), .)))

ERROR: Error in eval(lhs, parent, parent): object 'merged' not found


In [ ]:
#write.csv(merged_final, '../data/census_features/censusFeature_mean.csv')
# write.csv(merged_final, '../data/census_features/censusFeature.csv')